In [1]:
import numpy as np
import tensorly as tl
from copy import deepcopy

In [98]:
k = 2
dim = 20

# Create symmetric T with random vectors
#w = np.random.uniform(size=k)
w = np.ones(k)
w /= w.sum()
v1 = np.random.normal(size=(dim)) + 1
v2 = np.random.normal(size=(dim)) + 2
vs = np.stack([v1, v2])

# Compute the weighted outer product 
T = np.einsum('i,ij,ik,il->jkl', w, vs, vs, vs)
print(T.shape)

(20, 20, 20)


In [99]:
# Init
weights_init, factors_init = tl.decomposition._nn_cp.initialize_nn_cp(
    T, init='random', rank=k)
cp_init = tl.cp_tensor.CPTensor((weights_init, factors_init))

# Decomposition
weights, factors = tl.decomposition.parafac_power_iteration(T, rank=2)

T_hat_cp = tl.cp_tensor.CPTensor((weights, factors))
T_hat_cp_norm = tl.cp_tensor.cp_normalize(T_hat_cp)

weights = T_hat_cp_norm.weights


print(f'reconstructed weights:\t{np.sort(weights / weights.sum())}')
print(f'original weights:\t{np.sort(w)}')

T_hat = tl.cp_to_tensor(T_hat_cp_norm)
((T - T_hat)**2).mean()




reconstructed weights:	[0.2140798 0.7859202]
original weights:	[0.5 0.5]


0.6204271293981556

In [100]:
T[:2, :2, :2]

array([[[ 4.13753030e+00, -2.12507773e-01],
        [-2.12507773e-01,  1.56710487e-02]],

       [[-2.12507773e-01,  1.56710487e-02],
        [ 1.56710487e-02, -1.67844624e-03]]])

In [101]:
T_hat[:2, :2, :2]

array([[[ 4.14238430e+00, -2.15751402e-01],
        [-2.15751402e-01,  1.73143199e-02]],

       [[-2.15751402e-01,  1.73143199e-02],
        [ 1.73143199e-02, -1.73132083e-03]]])

In [62]:
# Power iteration

weights, factors = tl.decomposition.parafac_power_iteration(T, rank=k)

print(f'reconstructed weights:\t{np.sort(weights) / weights.sum()}')
print(f'original weights:\t{np.sort(w)}')

reconstructed weights:	[0.01225734 0.98774266]
original weights:	[0.34335817 0.65664183]


In [72]:
tensor = tl.random.random_cp(shape=(10, 10, 10), rank=2)
tensor

(weights, factors) : rank-2 CPTensor of shape (10, 10, 10) 